In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install mxnet
!pip install sentencepiece==0.1.96
!pip install gluonnlp==0.9.2
!pip install tokenizers

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 구글 드라이브와 연결 

Mounted at /content/drive


In [ ]:
import pandas as pd

chatbot_data_1 = '/content/drive/MyDrive/식품_train.csv'
chatbot_data_2 = '/content/drive/MyDrive/음식점_train.csv'
chatbot_data_3 = '/content/drive/MyDrive/카페.csv'

data_frame = pd.concat(
   map(pd.read_csv, [chatbot_data_3]), ignore_index=True)

In [ ]:
len(data_frame) 

21621

In [ ]:
set(data_frame['인텐트'])
from collections import defaultdict
intent_dict = defaultdict(list)


for intent in set(data_frame['인텐트']):
  # 주문 요청 
  if '주문' in intent and '요청' in intent:
    intent_dict[0].append(intent)
  elif '결제' in intent and '요청' in intent:
    intent_dict[1].append(intent)
  elif '적립' in intent and '요청' in intent:
    intent_dict[2].append(intent)
  elif '질문' in intent:
    intent_dict[3].append(intent)
  elif '확인' in intent:
    intent_dict[4].append(intent)
  elif '비교' in intent:
    intent_dict[5].append(intent)
  else:
    intent_dict[6].append(intent)

In [ ]:
# 주문 요청
data_frame.loc[(data_frame['인텐트'].isin(intent_dict[0])), '인텐트'] = 0

# 결제 요청
data_frame.loc[(data_frame['인텐트'].isin(intent_dict[1])), '인텐트'] = 1

# 적립 요청
data_frame.loc[(data_frame['인텐트'].isin(intent_dict[2])), '인텐트'] = 2

# 질문
data_frame.loc[(data_frame['인텐트'].isin(intent_dict[3])), '인텐트'] = 3

# 확인
data_frame.loc[(data_frame['인텐트'].isin(intent_dict[4])), '인텐트'] = 4

# 비교
data_frame.loc[(data_frame['인텐트'].isin(intent_dict[5])), '인텐트'] = 5

# 기타
data_frame.loc[(data_frame['인텐트'].isin(intent_dict[6])), '인텐트'] = 6

data_list = []
for q, label in zip(data_frame['발화문'], data_frame['인텐트']):
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

['케이크에서 약간 시큼한 맛이 나는데 상한건가요?', '3']
['라테 마신 컵에 아메리카노 리필 가능한가요?', '0']
['네, 하트파이 재고 남아있습니다.', '3']
['실온에 두면 안되는 거죠?', '4']


In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import gluonnlp as nlp


from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device("cuda:0")

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0)

tok = tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   ##클래스 수 조정##
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기

model = BERTClassifier(bertmodel, dr_rate=0.5).cuda()

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().cuda()
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().cuda()
        out = model(token_ids, valid_length, segment_ids).cuda()
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-43-6eed3988cb5d>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/305 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0003669261932373 train acc 0.140625
epoch 1 batch id 201 loss 0.6889327168464661 train acc 0.5367692786069652
epoch 1 train acc 0.5903176229508197


<ipython-input-43-6eed3988cb5d>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/34 [00:00<?, ?it/s]

...

epoch 9 test acc 0.7929462226066898


  0%|          | 0/305 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.26925647258758545 train acc 0.921875
epoch 10 batch id 201 loss 0.039900440722703934 train acc 0.9560789800995025
epoch 10 train acc 0.9570184426229508


  0%|          | 0/34 [00:00<?, ?it/s]

epoch 10 test acc 0.7961631343713956


In [ ]:
torch.save(model, "./drive/MyDrive/model.pth")
torch.save(model.state_dict(), "./drive/MyDrive/model.pth")
def predict(predict_sentence):
   
    data = [predict_sentence, '0']
    dataset_another = [data]
    #error point
    another_test = BERTDataset(dataset_another, 0, 1, tokenizer.tokenize, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("주문 요청")
            elif np.argmax(logits) == 1:
                test_eval.append("결제 요청")
            elif np.argmax(logits) == 2:
                test_eval.append("적립 요청")
            elif np.argmax(logits) == 3:
                test_eval.append("질문")
            elif np.argmax(logits) == 4:
                test_eval.append("확인")
            elif np.argmax(logits) == 5:
                test_eval.append("비교")
            elif np.argmax(logits) == 6:
                test_eval.append("기타")

        print(test_eval[0])

In [ ]:
# 질문 무한루프
while True:
    sentence = input("질문을 입력해주세요: ")
    if sentence == 'quit' :
        break
    predict(sentence)
    print("\n")

질문을 입력해주세요: 김치찌개 있나요?
질문


질문을 입력해주세요: 네이버페이로 결제할게요
결제 요청


질문을 입력해주세요: 김치찌개 하나요
주문 요청


질문을 입력해주세요: 포인트 적립할게요
적립 요청


질문을 입력해주세요: 마카롱에 뭐들어가요?
질문


질문을 입력해주세요: 둘중에 더 맛있는게 뭔가요?
질문


질문을 입력해주세요: quit
